In [317]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [318]:
import pysm3
import pysm3.units as u
from astropy import units as u
import pysm3.units as u
import numpy as np
import matplotlib.pyplot as plt
from fgbuster import CMB, Dust, Synchrotron, MixingMatrix
from fgbuster.observation_helpers import standardize_instrument, get_observation,  get_instrument
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline


In [319]:
nside = 64
instrument = get_instrument('LiteBIRD')

freq_mapsQ = get_observation(instrument, 'd0s0',nside=nside)[:,1,:]
freq_mapsU = get_observation(instrument, 'd0s0',nside=nside)[:,2,:]
freq_maps = get_observation(instrument, 'd0s0',nside=nside)

freq_mapsP = np.sqrt(np.power(freq_mapsQ,2)+np.power(freq_mapsU,2))


In [320]:
components= [CMB(),Dust(50.),Synchrotron(50.)]
A = MixingMatrix(*components)
A_ev = A.evaluator(instrument.frequency)


In [321]:
invN=np.eye(15)
invNd = np.einsum('ij,jsp->isp', invN, freq_maps)
invNd_P = np.einsum('ij,jp->ip', invN, freq_mapsP)

#invNd_P = invN.dot(freq_mapsP)

In [322]:
def spectral_likelihoodP(x):
    A_maxL = A_ev(x) 
    logL = 0
    AtNd= np.einsum('ji,jp->ip', A_maxL, invNd_P)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('ip,ij,jp->', AtNd, AtNA, AtNd)
    print(x, logL)
    if logL != logL:
        return 0.0
    return -logL



In [323]:
x0 = np.array([1.54,20,-3])
max= minimize(spectral_likelihoodP,x0)
max.x

[ 1.54 20.   -3.  ] 12748176585.85559
[ 1.54000001 20.         -3.        ] 12748176585.849901
[ 1.54       20.00000001 -3.        ] 12748176585.855415
[ 1.54       20.         -2.99999999] 12748176585.855244
[ 0.53235198 19.96892269 -3.06147903] 12701203927.85339
[ 0.532352   19.96892269 -3.06147903] 12701203929.068554
[ 0.53235198 19.9689227  -3.06147903] 12701203927.898205
[ 0.53235198 19.96892269 -3.06147901] 12701203927.939692
[ 1.53669984 19.99989822 -3.00020135] 12748177316.450985
[ 1.53669985 19.99989822 -3.00020135] 12748177316.450008
[ 1.53669984 19.99989823 -3.00020135] 12748177316.450903
[ 1.53669984 19.99989822 -3.00020134] 12748177316.450727
[ 1.53878536 19.97734333 -3.08633198] 12748178266.756836
[ 1.53878537 19.97734333 -3.08633198] 12748178266.755768
[ 1.53878536 19.97734335 -3.08633198] 12748178266.756723
[ 1.53878536 19.97734333 -3.08633196] 12748178266.756716
[ 1.54485463 19.72185599 -3.13906297] 12748178184.22746
[ 1.54485464 19.72185599 -3.13906297] 12748178184.22

array([ 1.54328503, 19.83057854, -3.13602678])

In [324]:

def spectral_likelihood(x):
    A_maxL = A_ev(x) 
    logL = 0
    AtNd= np.einsum('ji,jsp->isp', A_maxL, invNd)
    AtNA = np.linalg.inv(A_maxL.T.dot(invN).dot(A_maxL))
    logL = logL + np.einsum('isp,ij,jsp->', AtNd, AtNA, AtNd)
    print(x, logL)
    if logL != logL:
        return 0.0
    return -logL

In [325]:
x0 = np.array([1.54,20,-3])
max= minimize(spectral_likelihood,x0)
max.x

[ 1.54 20.   -3.  ] 27521315204038.234
[ 1.54000001 20.         -3.        ] 27521315204038.133
[ 1.54       20.00000001 -3.        ] 27521315204038.164
[ 1.54       20.         -2.99999999] 27521315204038.117
[ 0.93755428 19.5829222  -3.69512967] 27478704312530.54
[ 0.9375543  19.5829222  -3.69512967] 27478704314558.29
[ 0.93755428 19.58292221 -3.69512967] 27478704312597.234
[ 0.93755428 19.5829222  -3.69512966] 27478704312577.01
[ 1.53991768 19.99994301 -3.00009499] 27521315203305.12
[ 1.53991769 19.99994301 -3.00009499] 27521315203305.457
[ 1.53991768 19.99994302 -3.00009499] 27521315203305.26
[ 1.53991768 19.99994301 -3.00009497] 27521315203305.184
[ 1.53997339 19.99998158 -3.0000307 ] 27521315203962.805
[ 1.5399734  19.99998158 -3.0000307 ] 27521315203962.918
[ 1.53997339 19.99998159 -3.0000307 ] 27521315203962.78
[ 1.53997339 19.99998158 -3.00003069] 27521315203962.7
[ 1.5399934  19.99999543 -3.00000762] 27521315204033.836
[ 1.53999342 19.99999543 -3.00000762] 27521315204033.875


array([ 1.53999915, 19.99999941, -3.00000098])

In [326]:
instr = np.load('instrument_LB_IMOv1.npy', allow_pickle=True).item()
instr_ = {}
instr_['frequency'] = np.array([instr[f]['freq'] for f in instr.keys()])
instr_['depth_p'] = np.array([instr[f]['P_sens'] for f in instr.keys()])
instr_['fwhm'] = np.array([instr[f]['beam'] for f in instr.keys()])
instr_['depth_i'] = instr_['depth_p']/np.sqrt(2)
instr_standardize = standardize_instrument(instr_)

#print(instr_standardize)
print(instr)




{'LB_LFT_40': {'freq': 40.0, 'freq_band': 12.0, 'beam': 70.5, 'P_sens': 37.42}, 'LB_LFT_50': {'freq': 50.0, 'freq_band': 15.0, 'beam': 58.5, 'P_sens': 33.46}, 'LB_LFT_60': {'freq': 60.0, 'freq_band': 14.0, 'beam': 51.1, 'P_sens': 21.31}, 'LB_LFT_68a': {'freq': 68.0, 'freq_band': 16.0, 'beam': 41.6, 'P_sens': 19.91}, 'LB_LFT_68b': {'freq': 68.0, 'freq_band': 16.0, 'beam': 47.1, 'P_sens': 31.76}, 'LB_LFT_78a': {'freq': 78.0, 'freq_band': 18.0, 'beam': 36.9, 'P_sens': 15.56}, 'LB_LFT_78b': {'freq': 78.0, 'freq_band': 18.0, 'beam': 43.8, 'P_sens': 19.14}, 'LB_LFT_89a': {'freq': 89.0, 'freq_band': 20.0, 'beam': 33.0, 'P_sens': 12.28}, 'LB_LFT_89b': {'freq': 89.0, 'freq_band': 20.0, 'beam': 41.5, 'P_sens': 28.77}, 'LB_LFT_100': {'freq': 100.0, 'freq_band': 23.0, 'beam': 30.2, 'P_sens': 10.34}, 'LB_LFT_119': {'freq': 119.0, 'freq_band': 36.0, 'beam': 26.3, 'P_sens': 7.69}, 'LB_LFT_140': {'freq': 140.0, 'freq_band': 42.0, 'beam': 23.7, 'P_sens': 7.24}, 'LB_MFT_100': {'freq': 100.0, 'freq_band'